<a href="https://colab.research.google.com/github/AnnJungChan/youngjin_TECH/blob/main/DeepCandle/no1_process1_candle_chart_data_by_1hours.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 패키지로딩

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA 
from sklearn.preprocessing import MinMaxScaler

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# 데이터 로딩

- 1호기 1프로세스 대상으로 실행한다.
- 필요한 변수만 남기고 나머지는 모두 제거한 후, PCA를 진행한다.

In [2]:
no1_process1=pd.read_csv("/content/gdrive/MyDrive/용접데이터/1호기_프로세스1_scaled.csv")
no1_process1.drop(columns = ['Unnamed: 0','process','model_name'],inplace = True)
no1_process1=no1_process1.sort_values('date_time')
no1_process1=no1_process1.reset_index()
no1_process1.drop(columns = ['index'],inplace = True)

In [3]:
data = no1_process1[['A','V','gas','sec']]
data

,A,V,gas,sec
0,-0.239606,-0.173739,-0.390693,1.459233
1,-0.239606,-0.108440,-0.400172,1.250545
2,-0.212932,-0.124765,-0.409650,1.459233
3,-0.212932,-0.132927,-0.409650,1.459233
4,-0.239606,0.005833,-0.352778,1.459233
...,...,...,...,...
402836,0.213847,-0.075790,-0.561309,-1.253718
402837,0.213847,-0.116602,-0.561309,-1.253718
402838,0.213847,-0.075790,-0.542352,-1.253718
402839,0.213847,0.128268,-0.532873,-1.253718


# 캔들차트를 위한 PCA

- 캔들차트를 그리기 위해서는 1개~2개의 변수를 이용해야한다.
- 그러므로, 주성분 분석을 통해서 변수를 2개로 줄여주었다.

In [4]:
pca = PCA(n_components=4)
pca_model = pca.fit(data)
pca_data=pca.fit_transform(data)
print("설명력   \n" ,pca_model.explained_variance_ratio_)
print("상관계수\n", pca_model.components_)
print("데이터 \n " ,pca_data)

설명력   
 [0.44558322 0.42140011 0.08687227 0.04614439]
상관계수
 [[ 0.63822143 -0.28648393 -0.4049732  -0.58872495]
 [ 0.30575258  0.6476647   0.58248694 -0.38438881]
 [-0.38614669 -0.57922124  0.52029744 -0.4946555 ]
 [-0.59167516  0.40368323 -0.47539063 -0.51084646]]
데이터 
  [[-0.80401534 -0.9742712  -0.73193813 -0.4880787 ]
 [-0.69602361 -0.85728311 -0.67146335 -0.35060478]
 [-0.79334468 -0.94543924 -0.78046838 -0.47507873]
 ...
 [ 1.11592763  0.18230001  0.29929761  0.7411631 ]
 [ 1.05362948  0.31998283  0.18603418  0.81903212]
 [ 1.11785401  0.54223076  0.1326338   1.03800824]]


In [5]:
pca_dataframe=pd.DataFrame(pca_data)
pca_pc1=pca_dataframe[0]
pca_pc2=pca_dataframe[1]
print("주성분 1 \n" ,pca_pc1)
print("주성분 2 \n" ,pca_pc2)

주성분 1 
 0        -0.804015
1        -0.696024
2        -0.793345
3        -0.791006
4        -0.870814
            ...   
402836    1.123605
402837    1.135297
402838    1.115928
402839    1.053629
402840    1.117854
Name: 0, Length: 402841, dtype: float64
주성분 2 
 0        -0.974271
1        -0.857283
2        -0.945439
3        -0.950726
4        -0.835884
            ...   
402836    0.171258
402837    0.144825
402838    0.182300
402839    0.319983
402840    0.542231
Name: 1, Length: 402841, dtype: float64


# 캔들차트를 위한 데이터 전처리

- 캔들차트를 그리기 위해서 1시간 간격으로 데이터를 전처리함.

## 날짜 계산

- 날짜를 정확히 계산하여, 1시간 간격으로 데이터를 자르고 전처리할 준비를 함.

In [6]:
pca_pc1 = pd.concat([no1_process1[["date","time","date_time","decision"]],pd.DataFrame(pca_pc1)],axis = 1)
pca_pc2 = pd.concat([no1_process1[["date","time","date_time","decision"]],pd.DataFrame(pca_pc2)],axis = 1)
pca_pc1.rename(columns = {0:'pc1'},inplace = True)
pca_pc2.rename(columns = {1:'pc2'},inplace = True)

In [7]:
pca_pc1=pca_pc1[pca_pc1['date']<='2021-10-29']
pca_pc2=pca_pc2[pca_pc2['date']<='2021-10-29']

2월 4일부터 10월 29일까지 진행

In [8]:
print('최소날짜 : ', pca_pc1['date'].min())
print('최대날짜 : ', pca_pc1['date'].max())
print('최소시간 : ' , pca_pc1['time'].min())
print('최대시간 : ' , pca_pc1['time'].max())

최소날짜 :  2021-02-04
최대날짜 :  2021-10-29
최소시간 :  07:48:35
최대시간 :  20:55:37


In [9]:
import datetime
import time

time = datetime.datetime(2021,2,4,7,0,0)
print("날짜 차이 : " ,datetime.datetime(2021,10,29) - datetime.datetime(2021,2,4))

days = []
days.append(time)
for i in range(267) : 
  days.append(days[i] + datetime.timedelta(days = 1))

날짜 차이 :  267 days, 0:00:00


In [10]:
start_days = days
end_days = []
for i in range(len(days)) :
  end_days.append(days[i] + datetime.timedelta(hours = 14))

In [11]:
time_hours = []
for j in range(len(start_days)) :
  for i in range(15) :
    time_hours.append(start_days[j] + datetime.timedelta(hours = i))

In [12]:
time_str = []
for i in range(len(time_hours)) :
  time_str.append(time_hours[i].strftime('%Y-%m-%d %H:%M:%S'))

time_str이 시간 축이 되어 left로 데이터를 붙이는 역할을 하고, 값이 비었다면 time_str값을 제거시켜주면서 진행할 것이다.

## PC1에서 진행

In [71]:
top = []
bottom = []
min = []
max = []
volume = []


time = []

for i in range(len(time_str)) :
  temp=pca_pc1.loc[(time_str[i]<=pca_pc1['date_time']) & (time_str[i+1]>=pca_pc1['date_time']),"pc1"]
  if len(temp) != 0 :
    time.append(time_str[i])


for i in range(len(time)) :
  temp=pca_pc1.loc[(time[i]<=pca_pc1['date_time']) & (time[i+1]>=pca_pc1['date_time']),"pc1"]
  top.append(temp.head(1).values)
  bottom.append(temp.tail(1).values)
  min.append(temp.min())
  max.append(temp.max())
  volume.append(len(temp))

IndexError: ignored

In [72]:
print("time 길이 : ", len(time))
print("top 길이 : ", len(top))
print("bottom 길이 : ", len(bottom))
print("min 길이 : ", len(min))
print("max 길이 : ", len(max))
print("volume 길이 : ", len(volume))

time 길이 :  1835
top 길이 :  1834
bottom 길이 :  1834
min 길이 :  1834
max 길이 :  1834
volume 길이 :  1834


In [74]:
time = time[:1834]
data=pd.DataFrame({'time' : time, 'top' : top, 'bottom' : bottom, "min" : min, "max" : max, "volume" : volume })

In [81]:
data['top']=data['top'].astype(float)
data['bottom']=data['bottom'].astype(float)
data['min']=data['min'].astype(float)
data['max']=data['max'].astype(float)
data['volume']=data['volume'].astype(int)
data['time'] = data['time'].astype('datetime64')
data.to_csv("/content/gdrive/MyDrive/용접데이터/캔들차트분석/1호기 프로세스1/1hours_pc1.csv")

## PC2에서 진행

In [83]:
top = []
bottom = []
min = []
max = []
volume = []


time = []

for i in range(len(time_str)) :
  temp=pca_pc2.loc[(time_str[i]<=pca_pc2['date_time']) & (time_str[i+1]>=pca_pc2['date_time']),"pc2"]
  if len(temp) != 0 :
    time.append(time_str[i])


for i in range(len(time)) :
  temp=pca_pc2.loc[(time[i]<=pca_pc2['date_time']) & (time[i+1]>=pca_pc2['date_time']),"pc2"]
  top.append(temp.head(1).values)
  bottom.append(temp.tail(1).values)
  min.append(temp.min())
  max.append(temp.max())
  volume.append(len(temp))

IndexError: ignored

In [84]:
print("time 길이 : ", len(time))
print("top 길이 : ", len(top))
print("bottom 길이 : ", len(bottom))
print("min 길이 : ", len(min))
print("max 길이 : ", len(max))
print("volume 길이 : ", len(volume))

time 길이 :  1835
top 길이 :  1834
bottom 길이 :  1834
min 길이 :  1834
max 길이 :  1834
volume 길이 :  1834


In [85]:
time = time[:1834]
data=pd.DataFrame({'time' : time, 'top' : top, 'bottom' : bottom, "min" : min, "max" : max, "volume" : volume })
data['top']=data['top'].astype(float)
data['bottom']=data['bottom'].astype(float)
data['min']=data['min'].astype(float)
data['max']=data['max'].astype(float)
data['volume']=data['volume'].astype(int)
data['time'] = data['time'].astype('datetime64')
data.to_csv("/content/gdrive/MyDrive/용접데이터/캔들차트분석/1호기 프로세스1/1hours_pc2.csv")